In [0]:
## ok ##


with open(os.path.join(data_dir, "train.vocab"), "r") as _f_handle:
    vocab = [l.strip() for l in list(_f_handle) if len(l.strip()) > 0]
if len(vocab) > hparams.vocab_size:
    vocab = vocab[:hparams.vocab_size]

id2word = vocab
word2id = {}
for i, word in enumerate(vocab):
    word2id[word] = i


In [0]:
## ok ##

with open(os.path.join(data_dir, "label.vocab"), "r") as _f_handle:
  labels = [l.strip() for l in list(_f_handle) if len(l.strip()) > 0]
  labels.insert(0, "PAD")
  id2label = labels
  label2id = {}
  for i, label in enumerate(labels):
      label2id[label] = i

In [0]:
## ok ##

with tf.variable_scope("bi-RNN"):
    # Build RNN layers
    rnn_cell_forward = tf.contrib.rnn.LSTMCell(hparams.rnn_hidden_dim)
    rnn_cell_backward = tf.contrib.rnn.LSTMCell(hparams.rnn_hidden_dim)

    # Apply dropout to RNN
    if hparams.dropout_keep_prob < 1.0:
        rnn_cell_forward = tf.contrib.rnn.DropoutWrapper(rnn_cell_forward, output_keep_prob=dropout_keep_prob_ph)
        rnn_cell_backward = tf.contrib.rnn.DropoutWrapper(rnn_cell_backward, output_keep_prob=dropout_keep_prob_ph)

    # Stack multiple layers of RNN
    # rnn_cell_forward = tf.contrib.rnn.MultiRNNCell([rnn_cell_forward] * hparams.rnn_depth)
    # rnn_cell_backward = tf.contrib.rnn.MultiRNNCell([rnn_cell_backward] * hparams.rnn_depth)

    (output_forward, output_backward), _ = tf.nn.bidirectional_dynamic_rnn(
        rnn_cell_forward, rnn_cell_backward,
        inputs=layer_out,
        sequence_length=lengths,
        dtype=tf.float32
    )
    hiddens = tf.concat([output_forward, output_backward], axis=-1)
    # shape = [batch_size, time, rnn_dim*2]


In [0]:
## ok ##

sentence = word_tokenize(sentence)
word_ids = []
(id2word, word2id), (id2label, label2id) = make_vocab_table()

for word in sentence:
    if word in word2id:
        word_ids.append(word2id[word])
    else:
        word_ids.append(len(word2id))


In [0]:
## ok ##

with tf.variable_scope("read-out"):
  prev_layer_size = layer_out.get_shape().as_list()[1]
  weight = tf.get_variable("weight", shape=[prev_layer_size, output_dim],
                           initializer=tf.initializers.variance_scaling(
                               scale=2.0, mode="fan_in", distribution="normal"
                           ))
  bias = tf.get_variable("bias", shape=[output_dim],
                         initializer=tf.initializers.zeros())
  predictions = tf.add(tf.matmul(layer_out, weight), bias, name='predictions')

 

In [0]:
## ok ##
loss_op = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels,
                                                         name="cross_entropy")
loss_op = tf.reduce_mean(loss_op, name='cross_entropy_mean')
train_op = tf.train.AdamOptimizer().minimize(loss_op, global_step=global_step)

eval = tf.nn.in_top_k(logits, labels, 1)
correct_count = tf.reduce_sum(tf.cast(eval, tf.int32))
accuracy = tf.divide(correct_count, tf.shape(labels)[0])


In [0]:
## ok ##


dense_word_ids = tf.constant(word_ids)
lengths = tf.constant(len(word_ids))
# Insert batch dimension.
dense_word_ids = tf.expand_dims(dense_word_ids, axis=0)
lengths = tf.expand_dims(lengths, axis=0)

with tf.variable_scope("build_graph", reuse=tf.AUTO_REUSE):
    logits = build_graph(dense_word_ids, lengths, id2word, id2label)
predictions = tf.argmax(logits, axis=1)


In [0]:
## ok ##

word2id = tf.contrib.lookup.index_table_from_tensor(
  mapping=tf.constant(id2word),
  num_oov_buckets=1,
  name="word2id"
)

label2id = tf.contrib.lookup.index_table_from_tensor(
  mapping=tf.constant(id2label),
  default_value=label2id["O"],
  name="label2id"
)


In [0]:
## ok ##

label_dataset = tf.data.TextLineDataset(os.path.join(data_dir, "train.labels"))
batched_label_dataset = label_dataset.batch(hparams.batch_size)
label_iterator = batched_label_dataset.make_initializable_iterator()
batch_label_str = label_iterator.get_next()
batch_label = tf.string_split(batch_label_str, " ")
label_ids = label2id.lookup(batch_label)
dense_label_ids = tf.sparse_tensor_to_dense(label_ids)
# shape = [batch_size, time]

mask = tf.sequence_mask(lengths)
dense_label_ids = tf.boolean_mask(dense_label_ids, mask)

iterator_initializers.append(input_iterator.initializer)
iterator_initializers.append(label_iterator.initializer)


In [0]:
## ok ##

logger.info("End of epoch %d." % (epochs_completed+1))
save_path = saver.save(sess, "saves/model.ckpt", global_step=global_step_val)
logger.info("Model saved at: %s" % save_path)

In [0]:
## ok ##

input_dataset = tf.data.TextLineDataset(os.path.join(data_dir, "train.inputs"))
batched_input_dataset = input_dataset.batch(hparams.batch_size)
input_iterator = batched_input_dataset.make_initializable_iterator()
batch_input = input_iterator.get_next()
batch_input.set_shape([hparams.batch_size])
words = tf.string_split(batch_input, " ")
word_ids = word2id.lookup(words)
dense_word_ids = tf.sparse_tensor_to_dense(word_ids)
# shape = [batch_size, time]


line_number = word_ids.indices[:, 0]
line_position = word_ids.indices[:, 1]
lengths = tf.segment_max(data=line_position,
                         segment_ids=line_number) + 1


In [0]:
## ok ##

try:
  accuracy_val, label_ids_val, loss_val, global_step_val, _ = sess.run(
      [accuracy, labels, loss_op, global_step, train_op],
      feed_dict={dropout_keep_prob_ph: hparams.dropout_keep_prob}
  )
  accuracy_mean += accuracy_val
  loss_mean += loss_val
  idx_cnt += 1
  if global_step_val % 50 == 0:
      accuracy_mean /= idx_cnt
      loss_mean /= idx_cnt
      logger.info("[Step %d] loss: %.4f, accuracy: %.2f%%" % (global_step_val, loss_mean, accuracy_mean * 100))
      accuracy_mean, loss_mean,idx_cnt = 0, 0, 0
except tf.errors.OutOfRangeError:
  # End of epoch.
  break

In [0]:
## ok ##

saver = tf.train.Saver()
saver.restore(sess, saved_file)
pred_val = sess.run(
    [predictions]
)[0]


In [0]:

layer_out = tf.layers.dense(
    inputs=layer_out,
    units=hparams.rnn_hidden_dim,
    activation=tf.nn.relu,
    kernel_initializer=tf.initializers.variance_scaling(
        scale=1.0, mode="fan_avg", distribution="normal"),
    name="input_projection"
)


In [0]:
## ok ## 

pred_str = [id2label[i] for i in pred_val]
for word, tag in zip(sentence, pred_str):
    print("%s[%s]" %(word, tag), end=' ')


In [0]:
## ok ## 
  
mask = tf.sequence_mask(lengths)
bi_lstm_out = tf.reshape(tf.boolean_mask(hiddens, mask), [-1, hparams.rnn_hidden_dim * 2])
layer_out = bi_lstm_out  # shape=[sum of seq length, 2*LSTM hidden layer size]

In [0]:
## ok ## 
  
  
# Number of possible output categories.
output_dim = len(id2label)
vocab_size = len(id2word) + 1
embeddings = tf.get_variable(
    "embeddings",
    shape=[vocab_size, hparams.embedding_dim],
    initializer=tf.initializers.variance_scaling(
        scale=1.0, mode="fan_out", distribution="uniform")
)
embedded = tf.nn.embedding_lookup(embeddings, inputs)
# shape = [batch_size, time, embed_dim]
layer_out = embedded
